In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [207]:
import random

n = 4

source = [i for i in range(n)]
# target = sorted(source, key=lambda _: random.random())
target = source
mapping = {k: v for k, v in zip(source, target)}

mapping[n] = n      # BOS
mapping[n+1] = n+1  # EOS

In [208]:
def genseq(N):
    a = [n]
    for _ in range(N-2):
        a.append(random.randint(0, n-1))
    a.append(n+1)
    return a, [mapping[i] for i in a]

genseq(5)

([4, 0, 2, 0, 5], [4, 0, 2, 0, 5])

In [209]:
import torch
import torch.nn.functional as F

source_corpus = []
target_corpus = []

for _ in range(1):
    a, b = genseq(20)
    source_corpus.append(a)
    target_corpus.append(b)
    
source_corpus = torch.LongTensor(source_corpus)
target_corpus = torch.LongTensor(target_corpus)

In [210]:
from model import Transformer
from torch.optim import AdamW

n = 4

model = Transformer(d_model=10, nhead=2, max_len=20, vocab_size=n+2,
                    encoder_layers=3, decoder_layers=3)

optim = AdamW(model.parameters(), lr=3e-4)

In [213]:
from tqdm import tqdm

num_steps = 1000
batch_size = 16
eval_after = 100

for i in tqdm(range(num_steps)):
    model.train()
    samples = torch.LongTensor([random.randint(0, 0) for _ in range(batch_size)])
    input_ids = source_corpus[samples]
    output_ids = target_corpus[samples]
    loss, n = model.step(input_ids, output_ids)
    
    optim.zero_grad()
    (loss / n).backward()
    optim.step()
    
    if (i + 1) % eval_after == 0:
        print(f'most recent loss: {loss / n}')
        # model.eval()
        # tot_loss = 0
        # for j in range(10):
        #     input_ids = source_corpus[900+10*j:910+10*j]
        #     output_ids = target_corpus[900+10*j:910+10*j]
        #     loss, _ = model.step(input_ids, output_ids)
        #     tot_loss += loss
        # print(f'eval loss: {tot_loss / (100 * 19)}')

 10%|█         | 102/1000 [00:06<00:53, 16.82it/s]

most recent loss: 0.2001218944787979


 20%|██        | 202/1000 [00:11<00:44, 18.07it/s]

most recent loss: 0.15626995265483856


 30%|███       | 302/1000 [00:18<00:39, 17.62it/s]

...

most recent loss: 0.08685314655303955


 90%|█████████ | 902/1000 [00:51<00:05, 18.31it/s]

most recent loss: 0.05950372666120529


100%|██████████| 1000/1000 [00:56<00:00, 17.63it/s]

most recent loss: 0.13522973656654358


In [158]:
samples = torch.LongTensor([random.randint(0, 900) for _ in range(1)])
input_ids = source_corpus[samples]
output_ids = target_corpus[samples]

print(input_ids, output_ids)

model.eval()
src, _ = model.get_src(input_ids)
tgt, _ = model.get_src(output_ids)

tensor([[4, 0, 3, 3, 0, 2, 0, 2, 0, 0, 1, 2, 2, 1, 2, 3, 2, 2, 2, 5]]) tensor([[4, 1, 2, 2, 1, 3, 1, 3, 1, 1, 0, 3, 3, 0, 3, 2, 3, 3, 3, 5]])


In [167]:
logits, *_ = model.forward(src, tgt, None, None)
tok_probs = F.log_softmax(logits, dim=-1)

In [174]:
print(tok_probs[:, :-1])

print(output_ids[:, 1:])

tensor([[[-7.8902, -1.0121, -1.5015, -0.8898, -8.5908, -6.0252],
         [-7.9686, -1.2025, -1.4492, -0.7727, -9.1448, -5.9631],
         [-8.0529, -1.1184, -1.8047, -0.6882, -8.8073, -5.1613],
         [-8.0570, -1.1205, -1.8135, -0.6841, -8.8544, -5.1502],
         [-7.9452, -1.2128, -1.4799, -0.7522, -9.2235, -5.7498],
         [-7.9342, -1.1543, -1.5565, -0.7583, -8.8332, -5.3166],
         [-7.9326, -1.2382, -1.4867, -0.7354, -9.2025, -5.4718],
         [-7.8941, -1.1338, -1.4937, -0.7992, -8.7992, -5.6722],
         [-7.8833, -1.2282, -1.4117, -0.7750, -9.2665, -6.0850],
         [-7.8807, -1.2219, -1.4319, -0.7688, -9.2790, -6.0129],
         [-7.6218, -1.0934, -1.7086, -0.7372, -9.1922, -5.3503],
         [-7.9144, -1.1737, -1.5436, -0.7512, -8.8901, -5.3264],
         [-7.9260, -1.1824, -1.5550, -0.7410, -8.7753, -5.2764],
         [-7.4092, -1.0954, -1.7947, -0.7090, -9.1086, -5.0235],
         [-7.8591, -1.1501, -1.4506, -0.8085, -8.8677, -5.8402],
         [-8.0378, -1.146

In [175]:
from model import xent

xent(
    tok_probs[:, :-1],
    F.one_hot(output_ids[:, 1:], num_classes=6),
    ignore=-1
)

(tensor(32.0574, grad_fn=<NegBackward0>), tensor(19))

In [166]:
F.softmax(logits, dim=-1).shape

torch.Size([1, 20, 6])